In [1]:
import torch
from torchvision import models
#from simplify import simplify
import time
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

/home/songyao.jin/apps/anaconda3/envs/MBZUAI/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = models.resnet18(pretrained=True)
# print('model all param name :', [name for name, _ in model.named_parameters()])
dummy_input = torch.zeros(64, 3, 224,224)  # Tensor shape is that of a standard input for the given model
def run_model(model, dummy_input):
    start = time.time()
    for _ in tqdm(range(10)):
        model(dummy_input)
    end = time.time()
    print("Time taken: ", end - start)

print("Original model")
run_model(model, dummy_input)

/home/songyao.jin/apps/anaconda3/envs/MBZUAI/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/songyao.jin/apps/anaconda3/envs/MBZUAI/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Original model


100%|██████████| 10/10 [00:10<00:00,  1.01s/it]

Time taken:  10.117710590362549


In [3]:
! pip install tqdm

In [10]:
model.half()
print("half model")
#run_model(model, dummy_input.half())

half model


In [11]:
model = models.resnet18(pretrained=True) # reset model
import torch.quantization

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Conv2d,torch.nn.Linear}, dtype=torch.qint8
)

print("Quantized model")
run_model(quantized_model, dummy_input)

Quantized model


100%|██████████| 10/10 [00:08<00:00,  1.13it/s]

Time taken:  8.843028783798218
